# Full Length Article using GPT-3 API


### Overview

Content generation is a hot topic and a viable usecase for Large Language Models LLMs like GPT.


### Free Scripts

This is part of a series of free python scripts. 

You can find all the scripts online at https://github.com/FrontAnalyticsInc/data-winners

Message me on twitter if you have any suggestions.


### About Me

My name is Alton Alexander. I am a Data Science consultant turned entreprenuer building SaaS tools for SEO.

Find more about my free scripts or ask me any question on twitter: @alton_lex

# Setup

Steps: 

1) You'll need an API key from OpenAI.

2) Paste the key in the cell below or add it as an environment variable. For best practices see https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety

3) Run the following locally or in a google colab environment

In [30]:
# Set your api key
# copy and paste your open api key as a varible
import os

%env OPENAI_API_KEY=sk-PASTEYOURKEYHERE

open_api_key = os.getenv("OPENAI_API_KEY")

env: OPENAI_API_KEY=sk-PASTEYOURKEYHERE


In [24]:
# install libraries
%pip install openai
%pip install ratelimit

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import openai
openai.api_key = open_api_key

# Initialize the Advanced Function for API

First, we are using rate limiting because if you are creating a lot of articles then Python is too fast for the OpenAI API. Rate limit will slow it down.

Functions:

- request_openapi = a simple wrapper that applies basic instruction based on request type
- autogen_article = loops over topic to create title, intro, options, 
- autogen_article_2 = loops over each option and creates the text under it

Options for request types include:

- title
- introduction
- options = h2 headers
- option = paragraphs under h2 header

In [26]:
from ratelimit import limits, sleep_and_retry

# set rate limit for the account
ONE_MINUTE = 59

@sleep_and_retry
@limits(calls=20, period=ONE_MINUTE)
def request_openapi( request_type, params, engine ):

    try:
        if request_type == 'title':
            response = openai.Edit.create(
              engine="text-davinci-edit-001",
              input=params['input'],
              instruction="Rewrite the title with correct grammar and propper capitalization",
              temperature=0.7,
              top_p=1
            )
            return response

        if request_type == 'introduction':
            response = openai.Completion.create(
              engine=params.get('engine',engine),
              prompt=params['prompt'],
              temperature=0.7,
              max_tokens=256*2,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
            )
            return response

        if request_type == 'options':
            response = openai.Completion.create(
              engine=params.get('engine',engine),
              prompt=params['prompt'],
              temperature=0.7,
              max_tokens=256,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
            )
            return response

        if request_type == 'option':

            response = openai.Completion.create(
              engine=params.get('engine',engine),
              prompt=params['prompt'],
              temperature=0.7,
              max_tokens=256*2,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
            )
            return response
    except Exception as e:
        # error
        print(e)
        raise Exception('API response: {}'.format(e))




def autogen_article( inputs, method='openai', engine = "text-davinci-002" ):
    """Creates title, intro, and section headers"""
    
    # INITIALIZE THE ARTICLE
    article = {
        'topic': inputs['prompt_variables']['topic'],
        'inputs': inputs,
        'segments':[]
    }
    
    # TITLE
    if "title" in inputs['prompt_templates']:
        params = {}
        params['input'] = inputs['prompt_templates']['title'] % inputs['prompt_variables']
        response = request_openapi('title',params, engine)
        article['title'] = response['choices'][0]['text'].strip()
    
    # INTRODUCTION
    if "intro" in inputs['prompt_templates']:
        params = {}
        params['prompt'] = inputs['prompt_templates']['intro'] % inputs['prompt_variables']
        if(inputs['prompt_templates'].get("additional_keywords_in_intro")):
            params['prompt'] = params['prompt'] + " Be sure to use these keywords: " + ",".join(inputs['prompt_templates'].get("additional_keywords_in_intro")) + "."
        response1 = request_openapi('introduction', params, engine)
        segment = {
            'raw': {
                'response': response1,
                'params': params
            },
            'title': "",
            'type': "introduction",
            'body': response1['choices'][0]['text'].replace("\n\n","\n").replace("\n\n","\n").strip()
        }
        article['intro'] = segment
    
    
    # second prompt to get a list OF OPTIONS
    if "options" in inputs['prompt_templates']:
        params = {}
        params['prompt_template'] = inputs['prompt_templates']['options']+"\n1."
        params['prompt_variables'] = inputs['prompt_variables']
        params['prompt'] = params['prompt_template'] % params['prompt_variables']
        response2 = request_openapi('options', params, engine)
        results = response2['choices'][0]['text']
        results = results.replace("\n\n","\n").replace("\n\n","\n").replace("\n\n","\n")
        results = results.strip().split("\n")

        # add back on 1. to the first one
        if "1. " not in results[0]:
            results[0] = "1. "+results[0]

        article['options'] = {
            'raw': {
                'prompt': params['prompt'],
                'response': response2,
                'params': params
            },
            'results': results
        }
    else:
        results = []
    return article, results
    
    
def autogen_article_2( inputs, article, engine='text-curie-001' ):
    """Loop over each section header, and create the body paragraphs for that header"""
    
    if ('options' not in article.keys()) or ('each_option' not in inputs['prompt_templates'].keys()):
        return article, None
    
    # options
    results = article['options']['results']
    
    listtype = None
    numeric = 0
    bullet = 0
    for result in results[0:3]:
        check = result[:1].isdigit() and result[1:3]==". "
        if check:
            numeric = numeric + 1
        if "-" in result[0:1]:
            bullet = bullet + 1

    if bullet >= 2:
        listtype = "bullet"
    if numeric == 3:
        listtype = "numbered"

    new_queries = []
    for result in results:

        if listtype == 'numbered':
            if ". " not in result:
                continue
            new_query = result.strip().split(". ")[1].strip("\.")
            new_queries.append(new_query)
        
        if listtype == 'bullet':
            if "-" not in result[0:1]:
                continue
            new_query = result.split("-")[1].strip().strip("\.")
            new_queries.append(new_query)

    # third prompt
    # do a max of n number of new_queries
    max_n = inputs['settings'].get("options_max_n")
    if max_n is None:
        max_n = random.randrange(3,7)
    
    for option in new_queries[0:min(len(new_queries), max_n)]:

        option = option.split("–")[0].strip()

        
        # for each of the options
        prompt_variables = inputs['prompt_variables']
        prompt_variables['option'] = option
        prompt_template = inputs['prompt_templates']['each_option']
        
        params = {}
        params['prompt'] = prompt_template % prompt_variables
        params['engine'] = engine
        #print(prompt)
        
        response = request_openapi('option', params, engine)

        # cleanup the auto response
        body = response['choices'][0]['text'].replace("\n\n","\n").replace("\n\n","\n").strip()
        body = body.replace("\n1.","\n\n1.").replace("\n\n\n","\n\n")

        # new segment
        segment = {
            'raw': {
                'prompt_template': prompt_template,
                'prompt_variables': prompt_variables,
                'prompt': params['prompt'],
                'response': response,
                'params': params
            },
            'title': option,
            'type': "option",
            'body': body
        }
        article['segments'].append(segment)


    return article, results


def preview( article ):
    
    
    # preview one
    print("#TITLE:", article['title'])
    print()
    print("INTRO:", article['intro']['body'])
    print()
    for option in article['options']['results']:
        print(option)
    for segment in article['segments']:

        print()
        print("##", segment['title'])
        print()
        print(segment['body'])
        print()

    print()


# Weight Loss Example

In [27]:
topic = "weight loss"

keywords = [
    'fat',
    'belly',
    'man'
]

In [28]:

inputs = {
    "prompt_variables":{
        "condition": topic,
        "location": 'belly',
        "treatment": "diet",
    },
    "prompt_templates":{
        "title": "What are the most common questions treating %(condition)s on their %(location)s with %(treatment)s",
        "intro": "Using two or more paragraphs and at least 600 words, describe what causes %(condition)s on their %(location)s and how to treat it with %(treatment)s.",
        "additional_keywords_in_intro" : keywords,
        "options": "The most common specific questions that patients have when considering %(treatment)s to treat %(condition)s on their %(location)s are:",
        "each_option": "Answer the following question regarding treating %(condition)s with %(treatment)s: %(option)s"
    },
    "settings":{
        "options_max_n": 10
    }
}
inputs["prompt_variables"]["topic"] = "Most popular questions patients have about "+inputs["prompt_variables"]["condition"]+" and how to treat it with "+inputs["prompt_variables"]["treatment"]+""

inputs

{'prompt_variables': {'condition': 'weight loss',
  'location': 'belly',
  'treatment': 'diet',
  'topic': 'Most popular questions patients have about weight loss and how to treat it with diet'},
 'prompt_templates': {'title': 'What are the most common questions treating %(condition)s on their %(location)s with %(treatment)s',
  'intro': 'Using two or more paragraphs and at least 600 words, describe what causes %(condition)s on their %(location)s and how to treat it with %(treatment)s.',
  'additional_keywords_in_intro': ['fat', 'belly', 'man'],
  'options': 'The most common specific questions that patients have when considering %(treatment)s to treat %(condition)s on their %(location)s are:',
  'each_option': 'Answer the following question regarding treating %(condition)s with %(treatment)s: %(option)s'},
 'settings': {'options_max_n': 10}}

## Article with curie

Total cost less than $0.05

In [29]:
article, results = autogen_article( inputs )
article, results = autogen_article_2( inputs, article )
preview(article)

#TITLE: What are the most common questions about losing belly weight with dieting?

INTRO: There are many different reasons why someone may lose weight on their belly. It could be due to a medical condition, such as celiac disease or Crohn's disease. It could also be a result of a change in diet, such as going on a strict diet to lose weight. In some cases, weight loss on the belly may be due to stress or anxiety.
If you are losing weight on your belly and you are not sure why, it is important to see a doctor. They can help to determine the cause and recommend the best course of treatment. If your weight loss is due to a change in diet, they may recommend that you make some changes to your diet, such as eating more fruits and vegetables and cutting back on processed foods. If your weight loss is due to stress or anxiety, they may recommend therapy or medication.

1. What are the best foods to eat to lose weight?
2. What are the worst foods to eat to lose weight?
3. What are the best ex

## Article with Davinci 3

Total cost less than ~$0.25

In [21]:
article, results = autogen_article( inputs, engine='text-davinci-003' )
article, results = autogen_article_2( inputs, article, engine='text-davinci-003' )
preview(article)

#TITLE: What are the most common questions treating weight loss on their belly with diet?

INTRO: When it comes to belly fat, men are often more prone to it than women. This is because men typically have more belly fat than women, due to higher levels of testosterone. Belly fat is often the most difficult to lose and can be detrimental to a man's overall health. It is important to understand the causes of belly fat in order to effectively treat and reduce it.
The most common cause of belly fat is an unhealthy diet. Eating too much of the wrong types of foods, such as fried and processed foods, can lead to an accumulation of excess fat in the abdominal area. Additionally, consuming too many calories can also contribute to an increase in belly fat. To reduce belly fat, it is important to reduce caloric intake and focus on eating healthy, nutrient-dense foods. Additionally, it is important to increase physical activity levels to help burn the excess fat.
Another cause of belly fat can be 